In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_excel('Online_Retail.xlsx')

In [3]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.shape

(541909, 8)

In [5]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
dtype: object

In [7]:
df['InvoiceNo'].unique()

array([536365, 536366, 536367, ..., 581585, 581586, 581587], dtype=object)

In [8]:
df['InvoiceNo'] = df['InvoiceNo'].astype('str') 
# conveting to str as Invoice No is considered unique customer info

In [9]:
df[df['InvoiceNo'].str.contains('C')]  
# C in the Invoice no.means the product is cancelled

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,2010-12-01 09:41:00,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,2011-12-09 09:57:00,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315.0,United Kingdom


In [10]:
#Dropping rows without invoice number
df.dropna(subset=['InvoiceNo'], axis=0, inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

# Removing the credit transaction with invoice numbers containing C
df = df[~df['InvoiceNo'].str.contains('C')]
# ~ except that data, or negation operation

In [11]:
import re

In [12]:
def normalize_product(s):
    s = s.lower()
    
    # Remove punctuation that is non word internal
    s = re.sub('\s\W', ' ', s)  # \\s identifies whitespace
    s = re.sub('\W\s', ' ', s)  # \\W identifies non alphanum chars(other than letter, number and underscore)
    
    s = re.sub('\s+', ' ', s)   # \\s+ --- + identifies any additional \s i.e whitespace
    
    return s

<>:5: DeprecationWarning: invalid escape sequence '\s'
<>:6: DeprecationWarning: invalid escape sequence '\W'
<>:8: DeprecationWarning: invalid escape sequence '\s'
<>:5: DeprecationWarning: invalid escape sequence '\s'
<>:6: DeprecationWarning: invalid escape sequence '\W'
<>:8: DeprecationWarning: invalid escape sequence '\s'
C:\Users\srika\AppData\Local\Temp\ipykernel_12528\369378662.py:5: DeprecationWarning: invalid escape sequence '\s'
  s = re.sub('\s\W', ' ', s)  # \\s identifies whitespace
C:\Users\srika\AppData\Local\Temp\ipykernel_12528\369378662.py:6: DeprecationWarning: invalid escape sequence '\W'
  s = re.sub('\W\s', ' ', s)  # \\W identifies non alphanum chars(other than letter, number and underscore)
C:\Users\srika\AppData\Local\Temp\ipykernel_12528\369378662.py:8: DeprecationWarning: invalid escape sequence '\s'
  s = re.sub('\s+', ' ', s)   # \\s+ --- + identifies any additional \s i.e whitespace


In [13]:
df.dropna(subset=['Description'], axis = 0, inplace = True)

In [14]:
df['Description'] = df['Description'].astype('str')

In [15]:
df['Products'] = [normalize_product(s) for s in df['Description']]

In [16]:
# Removing spaces from the description

# this step is similar to what we did above
# df['Description'] = df['Description'].str.strip()
# df.dropna(subset=['Description'],axis=0, inplace=True)
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Products
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,white hanging heart t-light holder
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,white metal lantern
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,cream cupid hearts coat hanger
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,knitted union flag hot water bottle
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,red woolly hottie white heart.
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,pack of 20 spaceboy napkins
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,children's apron dolly girl
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,childrens cutlery dolly girl
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,childrens cutlery circus parade


In [17]:
df[df['Products'].str.contains('wrong')]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Products
114522,546018,85172,wrongly sold as sets,-600,2011-03-08 17:23:00,0.0,NaN,United Kingdom,wrongly sold as sets
114538,546023,85175,wrongly sold sets,-975,2011-03-08 17:29:00,0.0,NaN,United Kingdom,wrongly sold sets
117894,546407,22719,wrong barcode (22467),-178,2011-03-11 16:24:00,0.0,NaN,United Kingdom,wrong barcode 22467)
117895,546408,22467,wrongly sold (22719) barcode,170,2011-03-11 16:25:00,0.0,NaN,United Kingdom,wrongly sold 22719 barcode
128464,547336,21689,wrong barcode,-323,2011-03-22 11:45:00,0.0,NaN,United Kingdom,wrong barcode
168324,551019,22467,wrong code?,-100,2011-04-26 11:19:00,0.0,NaN,United Kingdom,wrong code?
168325,551020,22719,wrong code,-110,2011-04-26 11:19:00,0.0,NaN,United Kingdom,wrong code
263884,560039,20713,wrongly marked. 23343 in box,-3100,2011-07-14 14:27:00,0.0,NaN,United Kingdom,wrongly marked 23343 in box
277345,561103,85103,stock creditted wrongly,-32,2011-07-25 11:23:00,0.0,NaN,United Kingdom,stock creditted wrongly
380687,569830,23343,wrongly coded 20713,800,2011-10-06 12:38:00,0.0,NaN,United Kingdom,wrongly coded 20713


In [18]:
# pd.DataFrame(df.groupby(['InvoiceNo', 'Products'])['Quantity'].sum())

# summing the quantity
# InvoiceNo Products in index
# notice Quantity higher position

#for this reason need to use reset_index()

In [19]:
# df.groupby(['InvoiceNo', 'Products'])['Quantity'].sum().unstack().reset_index().fillna(0)
# unstack() this is used to make Products as columns(here innermost column which we mentioned in the groupby)
# fillna(0) because after unstack, majority Nan will be there, so need to fill with 0

# the purpose of this is we are trying to have associacion rule, so we need item wise and user wise
# here invoiceNo is user and Products is the items

In [20]:
basket = df.pivot_table( values= 'Quantity', index="InvoiceNo", columns="Products", fill_value=0)
# this is also similar to above code

basket = (df
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum()
          .unstack()
          .fillna(0))

* this is same as above 2 times we performed

In [21]:
basket

Products,4 purple flock dinner candles,50's christmas gift bag large,dolly girl beaker,i love london mini backpack,i love london mini rucksack,nine drawer office tidy,oval wall mirror diamante,red spot gift bag large,set 2 tea towels i love london,sold as sets?,...,zinc star t-light holder,zinc sweetheart soap dish,zinc sweetheart wire letter rack,zinc t-light holder star large,zinc t-light holder stars large,zinc t-light holder stars small,zinc top 2 door wooden shelf,zinc willie winkie candle stick,zinc wire kitchen organiser,zinc wire sweetheart letter tray
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0,0.0,0,0,0,0,0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0
536366,0.0,0,0.0,0,0,0,0,0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0
536367,0.0,0,0.0,0,0,0,0,0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0
536368,0.0,0,0.0,0,0,0,0,0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0
536369,0.0,0,0.0,0,0,0,0,0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581586,0.0,0,0.0,0,0,0,0,0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0
581587,0.0,0,0.0,0,0,0,0,0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0
A563185,0.0,0,0.0,0,0,0,0,0,0.0,0,...,0,0,0,0.0,0,0,0,0.0,0,0


In [22]:
basket["pack of 20 spaceboy napkins"].unique()
# randomly take one column and check unique values to check

array([  0. ,   1. ,   3. ,   6. ,   2. ,  12. ,   5. ,   4. ,  96. ,
        24. ,   3.5, 192. ,  36. ,   8. ,  10. ])

In [23]:
# here we don't need the information of how many products a customer has purchased
# we only want whether they have purchased or not.
# for this reason we perform below action

In [24]:
def encode_units(x):
    if x <= 0:
        return 0    
    if x >= 1:
        return 1
    
basket_sets = basket.applymap(encode_units)
basket_sets.tail(10)

Products,4 purple flock dinner candles,50's christmas gift bag large,dolly girl beaker,i love london mini backpack,i love london mini rucksack,nine drawer office tidy,oval wall mirror diamante,red spot gift bag large,set 2 tea towels i love london,sold as sets?,...,zinc star t-light holder,zinc sweetheart soap dish,zinc sweetheart wire letter rack,zinc t-light holder star large,zinc t-light holder stars large,zinc t-light holder stars small,zinc top 2 door wooden shelf,zinc willie winkie candle stick,zinc wire kitchen organiser,zinc wire sweetheart letter tray
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
581581,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581582,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581583,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581584,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581585,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
581586,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
581587,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A563185,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A563186,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
basket_sets["pack of 20 spaceboy napkins"].unique()
# now you see only 0 and 1 unique values

array([0, 1], dtype=int64)

In [26]:
basket_sets.shape
# here 20610 unique customers
# 4171 unique products

(20610, 4171)

In [27]:
basket_sets.describe()

Products,4 purple flock dinner candles,50's christmas gift bag large,dolly girl beaker,i love london mini backpack,i love london mini rucksack,nine drawer office tidy,oval wall mirror diamante,red spot gift bag large,set 2 tea towels i love london,sold as sets?,...,zinc star t-light holder,zinc sweetheart soap dish,zinc sweetheart wire letter rack,zinc t-light holder star large,zinc t-light holder stars large,zinc t-light holder stars small,zinc top 2 door wooden shelf,zinc willie winkie candle stick,zinc wire kitchen organiser,zinc wire sweetheart letter tray
count,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000,20610.0,...,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000,20610.000000
mean,0.001844,0.006259,0.008443,0.004173,0.000049,0.001601,0.007618,0.004998,0.013052,0.0,...,0.002280,0.001019,0.004367,0.008103,0.000097,0.013295,0.000582,0.013052,0.000776,0.001116
std,0.042901,0.078868,0.091497,0.064463,0.006966,0.039983,0.086948,0.070518,0.113500,0.0,...,0.047701,0.031905,0.065939,0.089653,0.009851,0.114536,0.024123,0.113500,0.027852,0.033388
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
# Generating frequent item sets that have a support of at least 3%

frequent_itemsets = apriori(basket_sets, min_support=0.03, use_colnames=True)
frequent_itemsets

# min_support is threshold support value which we considered in association rule
# use_colnames will give me name of the column in following table result

C:\Users\srika\anaconda3\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
0,0.046337,(6 ribbons rustic charm)
1,0.040175,(60 teatime fairy cake cases)
2,0.047550,(alarm clock bakelike green)
3,0.037991,(alarm clock bakelike pink)
4,0.050995,(alarm clock bakelike red )
...,...,...
122,0.040029,"(jumbo bag pink polkadot, jumbo bag red retros..."
123,0.032994,"(jumbo shopper vintage red paisley, jumbo bag ..."
124,0.035129,"(jumbo storage bag suki, jumbo bag red retrospot)"
125,0.031101,"(lunch bag red retrospot, lunch bag black skull.)"


In [29]:
# Generating the rules with their corresponding support, confidence and lift

rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)

rules.head()

# min_threshold means what is the chance of purchasing 2nd product after buying 1st product
# recall that lift should be greated than 1, which means sure shot purchasing a product min 1 qty

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(alarm clock bakelike red ),(alarm clock bakelike green),0.050995,0.047550,0.031053,0.608944,12.806462,0.028628,2.435585,0.971453
1,(alarm clock bakelike green),(alarm clock bakelike red ),0.047550,0.050995,0.031053,0.653061,12.806462,0.028628,2.735368,0.967940
2,(green regency teacup and saucer),(pink regency teacup and saucer),0.049248,0.037166,0.030713,0.623645,16.779804,0.028883,2.558314,0.989116
3,(pink regency teacup and saucer),(green regency teacup and saucer),0.037166,0.049248,0.030713,0.826371,16.779804,0.028883,5.475760,0.976705
4,(green regency teacup and saucer),(roses regency teacup and saucer ),0.049248,0.051722,0.037263,0.756650,14.629045,0.034716,3.896768,0.979901


In [30]:
# filtering out the values with lift > = 6 and confidence >= 0.7

rules[(rules['lift'] >= 6) & (rules['lift'] >= 0.7)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(alarm clock bakelike red ),(alarm clock bakelike green),0.050995,0.047550,0.031053,0.608944,12.806462,0.028628,2.435585,0.971453
1,(alarm clock bakelike green),(alarm clock bakelike red ),0.047550,0.050995,0.031053,0.653061,12.806462,0.028628,2.735368,0.967940
2,(green regency teacup and saucer),(pink regency teacup and saucer),0.049248,0.037166,0.030713,0.623645,16.779804,0.028883,2.558314,0.989116
3,(pink regency teacup and saucer),(green regency teacup and saucer),0.037166,0.049248,0.030713,0.826371,16.779804,0.028883,5.475760,0.976705
4,(green regency teacup and saucer),(roses regency teacup and saucer ),0.049248,0.051722,0.037263,0.756650,14.629045,0.034716,3.896768,0.979901
5,(roses regency teacup and saucer ),(green regency teacup and saucer),0.051722,0.049248,0.037263,0.720450,14.629045,0.034716,3.401012,0.982458
6,(jumbo bag pink polkadot),(jumbo bag red retrospot),0.059098,0.101504,0.040029,0.677340,6.673028,0.034030,2.784651,0.903540
7,(jumbo bag red retrospot),(jumbo bag pink polkadot),0.101504,0.059098,0.040029,0.394359,6.673028,0.034030,1.553566,0.946185
10,(jumbo storage bag suki),(jumbo bag red retrospot),0.057448,0.101504,0.035129,0.611486,6.024253,0.029297,2.312650,0.884836
11,(jumbo bag red retrospot),(jumbo storage bag suki),0.101504,0.057448,0.035129,0.346080,6.024253,0.029297,1.441388,0.928223


In [31]:
# above meanings of columns

# antecedents = already purchased
# consequents = that they are going to purchase
# antecedent support = support value for antecedents
# consequent support = support value for consequent
# support = support betn both products
# confidence = as high as possible (confidence value)
# lift = 
# leverage = 
# conviction = 
# zhangs_metric = 

In [32]:
help(association_rules)

Help on function association_rules in module mlxtend.frequent_patterns.association_rules:

association_rules(df, metric='confidence', min_threshold=0.8, support_only=False)
    Generates a DataFrame of association rules including the
    metrics 'score', 'confidence', and 'lift'
    
    Parameters
    -----------
    df : pandas DataFrame
      pandas DataFrame of frequent itemsets
      with columns ['support', 'itemsets']
    
    metric : string (default: 'confidence')
      Metric to evaluate if a rule is of interest.
      **Automatically set to 'support' if `support_only=True`.**
      Otherwise, supported metrics are 'support', 'confidence', 'lift',
      'leverage', 'conviction' and 'zhangs_metric'
      These metrics are computed as follows:
    
      - support(A->C) = support(A+C) [aka 'support'], range: [0, 1]
    
      - confidence(A->C) = support(A+C) / support(A), range: [0, 1]
    
      - lift(A->C) = confidence(A->C) / support(C), range: [0, inf]
    
      - levera

In this apriori there is nothing like model building, here is just predicting the probability
and all the above pre processing steps to be done

END